In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib inline

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('./src')

from data_queries import  getClusterInfo, getGAIAKnownMembers
from gaiastars import gaiastars as gs

import astropy.units as u

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
from  gaiastars import gaiadr2xdr3
known_cluster_members, cluster_names = getGAIAKnownMembers()
# just deal with Pleiades and alphaPer for now
cluster_names = ['Pleiades', 'alphaPer']
xmatches = {}
cluster_members={}
#for cl in cluster_names:
for cl in cluster_names:
    known_members_dr2 = list(known_cluster_members.query('Cluster == @cl').index)
    xmatches[cl] = gaiadr2xdr3(known_members_dr2)
    cluster_members[cl]  = gs(name = cl, description=f'{cl} sources from Table 1a records from Gaia archive')
    cluster_members[cl].from_source_idlist(list(xmatches[cl].dr3_source_id),schema='gaiaedr3', query_type='sync')

#construct a dict mapping cluster name in Table1a to its name in Simabad
cluster_info = getClusterInfo()

search_results = {}

from gaiastars import from_pickle

for cl in cluster_names:
    search_results[cl] = from_pickle(f'./data/search_results_{cl}.pkl')

#exclude the known members returned from the search
for cl in cluster_names:
    merged_fs = search_results[cl].merge(cluster_members[cl])
    print(f'------ {cl} -------')
    print(merged_fs.objs.which.value_counts())
    fs = merged_fs.query('which == \'{} cone search\''.format(cl))
    fs.name = 'Search Results, Known Members excluded'
    search_results[cl] = fs

for cl in cluster_names:
    maxsep = cluster_members[cl].maxsep() /2.0
    print(f'{cl}: Maxsep: {maxsep}')
    search_results[cl].points_to(cluster_info.loc[cl]['coords'],maxsep.value, inplace=True, allcalcs=True)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
------ Pleiades -------
Pleiades cone search    107919
both                      1225
Pleiades                   101
Name: which, dtype: int64
------ alphaPer -------
alphaPer cone search    291664
both                       699
alphaPer                    41
Name: which, dtype: int64
Pleiades: Maxsep: 3.328984626197819 deg
alphaPer: Maxsep: 2.5454915501451394 deg


In [3]:
import os
os.getcwd()

'/home/kevin_leo_mcmanus/repos/runawaysearch'

In [4]:
maxsep = cluster_members['Pleiades'].maxsep() /2.0
maxsep

<Angle 3.32898463 deg>

In [5]:
from explain_pt import explain_pt

In [6]:
explain_pt(search_results['Pleiades'].objs.iloc[222], cluster_info.loc['Pleiades']['coords'], maxsep.degree)

Input:
               ra        dec  pm_ra_cosdec      pm_dec
Which                                                 
Star     44.54928   2.170299     77.164331 -120.947563
Cluster  56.44000  23.860000     19.997000  -45.548000

Colatitude Star (radian): 1.5329
Colatitude Center (radian): 1.1544
Polar Angle, Star-to-Center (radian): -0.2075

Step 1: Calculate Theta, angle from star to center wrt. Meridian
----------------------------------------------------------------
Step 1a: Calculate Great Circle Distance Star-to-Center (radian)
	cos(star_cen_dist) = cos(colat_star)*cos(colat_center) + sin(colat_star)*sin(colat_center)*cos(polar_angle)
	cos(star_cen_dist) = cos(1.5329)*cos(1.1544) + sin(1.5329)*sin(1.1544)*cos(-0.2075)
	cos(star_cen_dist) = 0.9096
	star_cen_dist = 0.4285 radian
Step 1b: Calculate theta: angle from star to center wrt. meridian
	cos(colat_center) = cos(star_cen_dist)*cos(colat_star) + sin(star_cen_dist)*sin(colat_star)*cos(theta)
	Solve for theta:
	cos(theta) = (cos(c

In [7]:
search_results['Pleiades'].objs.iloc[222]

_merge                         left_only
which               Pleiades cone search
ra                               44.5493
dec                               2.1703
parallax                          6.6092
pmra                             77.1643
pmdec                           -120.948
radial_velocity                      NaN
phot_g_mean_mag                  16.7115
phot_bp_mean_mag                 17.6973
phot_rp_mean_mag                 15.0903
ruwe                             1.14379
r_est                            151.304
cen_dist                        0.428502
theta                           0.470697
c                               0.432181
gamma                           0.139087
delta_pm_ra                      57.1673
delta_pm_dec                    -75.3996
pm_dir                           2.49287
pm_dir_prime                     5.63446
points_to                          False
Name: 1282756408103936, dtype: object

In [8]:
search_results['Pleiades'].objs.query('points_to').head()

,_merge,which,ra,dec,parallax,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_bp_mean_mag,...,r_est,cen_dist,theta,c,gamma,delta_pm_ra,delta_pm_dec,pm_dir,pm_dir_prime,points_to
source_id,,,,,,,,,,,,,,,,,,,,,
621228365371520,left_only,Pleiades cone search,46.443202,1.760904,6.709978,-13.442013,-133.510406,NaN,15.443704,16.635983,...,149.031779,0.421060,0.398914,0.424812,0.141363,-33.439013,-87.962406,-2.778314,0.363279,True
621297084847872,left_only,Pleiades cone search,46.439646,1.767968,6.726454,-13.464543,-133.151323,NaN,16.428431,17.805946,...,148.666739,0.420971,0.399147,0.424724,0.141391,-33.461543,-87.603323,-2.776728,0.364864,True
824294419098240,left_only,Pleiades cone search,47.210041,2.729455,12.909211,4.891285,-92.906115,NaN,14.395027,15.821107,...,77.464065,0.400326,0.385905,0.404295,0.148161,-15.105715,-47.358115,-2.832826,0.308766,True
911632578530944,left_only,Pleiades cone search,45.945342,2.521983,8.495969,10.539835,-70.220999,NaN,15.493484,16.859322,...,117.702884,0.412388,0.428617,0.416228,0.144122,-9.457165,-24.672999,-2.775565,0.366028,True
1021759835513088,left_only,Pleiades cone search,46.698325,2.766141,8.174193,-15.796483,-114.005202,NaN,17.052393,18.809666,...,122.336233,0.403174,0.405433,0.407112,0.147185,-35.793483,-68.457202,-2.659825,0.481767,True


In [9]:
explain_pt(search_results['Pleiades'].objs.loc[1021759835513088], cluster_info.loc['Pleiades']['coords'], maxsep.degree)

Input:
                ra        dec  pm_ra_cosdec      pm_dec
Which                                                  
Star     46.698325   2.766141    -15.796483 -114.005202
Cluster  56.440000  23.860000     19.997000  -45.548000

Colatitude Star (radian): 1.5225
Colatitude Center (radian): 1.1544
Polar Angle, Star-to-Center (radian): -0.1700

Step 1: Calculate Theta, angle from star to center wrt. Meridian
----------------------------------------------------------------
Step 1a: Calculate Great Circle Distance Star-to-Center (radian)
	cos(star_cen_dist) = cos(colat_star)*cos(colat_center) + sin(colat_star)*sin(colat_center)*cos(polar_angle)
	cos(star_cen_dist) = cos(1.5225)*cos(1.1544) + sin(1.5225)*sin(1.1544)*cos(-0.1700)
	cos(star_cen_dist) = 0.9198
	star_cen_dist = 0.4032 radian
Step 1b: Calculate theta: angle from star to center wrt. meridian
	cos(colat_center) = cos(star_cen_dist)*cos(colat_star) + sin(star_cen_dist)*sin(colat_star)*cos(theta)
	Solve for theta:
	cos(theta) = (c

In [10]:
search_results['Pleiades'].objs.loc[1021759835513088]

_merge                         left_only
which               Pleiades cone search
ra                               46.6983
dec                              2.76614
parallax                         8.17419
pmra                            -15.7965
pmdec                           -114.005
radial_velocity                      NaN
phot_g_mean_mag                  17.0524
phot_bp_mean_mag                 18.8097
phot_rp_mean_mag                 15.7962
ruwe                             1.05491
r_est                            122.336
cen_dist                        0.403174
theta                           0.405433
c                               0.407112
gamma                           0.147185
delta_pm_ra                     -35.7935
delta_pm_dec                    -68.4572
pm_dir                          -2.65983
pm_dir_prime                    0.481767
points_to                           True
Name: 1021759835513088, dtype: object

## Star RA > Center RA

In [11]:
cluster_info.loc['Pleiades']['coords']

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (56.44, 23.86, 135.79576317)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (19.997, -45.548, 5.65)>

In [12]:
search_results['Pleiades'].objs.query('ra > 56.44').tail()

,_merge,which,ra,dec,parallax,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_bp_mean_mag,...,r_est,cen_dist,theta,c,gamma,delta_pm_ra,delta_pm_dec,pm_dir,pm_dir_prime,points_to
source_id,,,,,,,,,,,,,,,,,,,,,
3449147738451308416,left_only,Pleiades cone search,82.352867,33.546045,5.532237,4.096252,-21.762671,NaN,20.338305,21.092461,...,180.758692,0.429651,1.856600,0.433319,0.138742,-15.900748,23.785329,-0.589278,2.552314,False
3449148223780771840,left_only,Pleiades cone search,82.455613,33.539545,5.369250,-3.505266,-0.527862,NaN,20.645977,20.577534,...,186.245762,0.431053,1.854457,0.434708,0.138324,-23.502266,45.020138,-0.481123,2.660470,False
3449150732044356736,left_only,Pleiades cone search,82.587119,33.551089,7.593882,-15.165790,-88.632166,NaN,13.191804,13.960330,...,131.684949,0.432947,1.852451,0.436583,0.137765,-35.162790,-43.084166,-2.457086,0.684506,False
3449158291184012160,left_only,Pleiades cone search,82.587722,33.783645,5.471653,6.113794,-2.158403,NaN,20.650612,20.344765,...,182.760141,0.434099,1.860853,0.437725,0.137426,-13.883206,43.389597,-0.309672,2.831920,False
3449182549161746560,left_only,Pleiades cone search,82.634579,33.855486,8.155049,-124.381307,-90.466599,NaN,16.208384,17.708820,...,122.623418,0.435110,1.862566,0.438726,0.137131,-144.378307,-44.918599,-1.872421,1.269172,False
